# Sensitivity, Specificity, Odd Ratios and other measures

In [1]:
# This cell is a code cell.  In cells like this
# we run Python code

# Here's some code that will likely appear near the top of every homework or lecture this semester.

from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import scipy.stats as stats

import scipy

import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=np.VisibleDeprecationWarning)

# To add text, referred to as comments to a code cell
# just put a hashtag a the beginning of the comment
# Comments are ignored by the computer when executing the code


mort22 = Table.read_table("SmallMort2022.csv")

## Sensitivity vs Specificity

Imagine a test to determine if a person is sick.  The sickness is such that eventually it will be obvious without the test whether the peron truly has it or not.  

Suppose for a reasonably sized group of people the following two-way table shows how many people were or were not sick split by their test results.  


|        |Test Pos. | Test Neg. |
|---     |:-:       |:-:        |
|Sick    |a        |b         |
|Not Sick|c        |d          |


Sensitivity is the True positive rate.  That is, if the test reads positive, what is the probability the person is sick.  

$$ Sensitivity = \frac{a}{a+c}$$


Specificity is the True negative rate.  That is, if the test reads negative, what is the probablity the person is NOT sick?

$$Specificity = \frac{d}{b+d}$$


Another way of defining these is as conditional probabilities.  Let $X = 1$ mean that the test is positive and $Y = 1$ mean that the person really is sick.  In both cases, 0 is used for the opposite.  Then,

$$ Sensitivity = Pr\left(Y = 1 | X = 1\right)$$

$$ Specificity = Pr\left(Y = 0 | X = 0 \right)$$


Consider this data about childrens health.  The Asthma variable records whether the child has been diagnosed with asthma and the Father variable records whether the childs father ever has.  


In [2]:
asthma = Table.read_table("asthma.csv")

asthma.select("Asthma", "Father")

Asthma,Father
No,No
No,No
No,No
No,No
No,No
No,No
No,No
No,No
No,No
No,No


In [3]:
asthma = Table.read_table("asthma.csv")

asthma.select("Asthma", "Father")

asthma.pivot("Father", "Asthma")

Asthma,No,Yes
No,11329,0
Yes,185,37


## False and True Positives and Negatives

You may encounter a source that defines sensitivity in terms of true positive and false negatives, and specificity in term of false positive and true negatives.  If those notions make more sense to you, please feel free to use them.  


|        |Test Pos. | Test Neg. |
|---     |:-:       |:-:        |
|Sick    |True Pos. |False Neg. |
|Not Sick|False Pos.|True Neg.   |



If, as a crude test, we simply used whether the father has asthma as a predictor for whether the child will, what would the specificity and sensitivity of that "test" be, according to these data?

In theory, we should be able to rearrange the numbers in our minds, but for our very first example, let's reorganize the so-called *confusion matrix* so that its format matches the layout we used in the defintions above.  

|      | Test Pos.  | Test Neg.  |
|--- |:-:|:-:|
|sick|37 | 185 |
|not | 0 |11329 |

So, 

$$ Sens. = \frac{a}{a+c} = \frac{37}{37+0} = \frac{37}{37} = 1$$

But,

$$ Spec. = \frac{d}{c+d} = \frac{11329}{11329 + 185} \approx 0.984 $$

So, as a crude test to determine whether a child may develop asthma just knowing if the father did is not that useful.  If the father did not, then it would seem that the child won't.  

This "test" has no False Positives, but it has a lot of False Negatives.  This example illustrates something that is often true; there is a trade-off between these things.  Usually, as Sensitivity goes up Specificity goes down (or vice versa).  

Next, consider this older data set relating whether an HIV positive patient took the drug AZT and whether that patient developed AIDS.  



In [4]:
11329/(185+11329)

0.9839326037866944

In [57]:
azt = Table.read_table("AIDS.csv" )

azt

race,azt,yes,no
white,yes,14,93
white,no,32,81
black,yes,11,52
black,no,12,43


In [58]:
azt2 = azt.drop("race").group("azt", sum).relabel(1,"Yes").relabel(2, "No")

azt2

azt,Yes,No
no,44,124
yes,25,145


In [6]:
145/(145+124)

0.5390334572490706

If we use whether the person accepted AZT as the "test" for whether he or she developed AIDS, what is the sensitivity and specificity of that "test"?


|      |Test Pos. (No AZT)|Test Neg. (Yes AZT)|
|---   |:-:      |:-:      |
|sick  |44       |124      |
|not   |25      |145       |


$$ Sens = \frac{44 }{25+44 }  \approx 0.638 $$


$$ Spec = \frac{145}{145+124} \approx 0.539 $$

## Other related and relevant terms

Of course, the terms "sensitivity" and "specificity" come from the medical industry, as such might not seem appropriate measures in some cases.  

The *relative risk* and *odds ratios* might be better measures.  

Let's see those defined first.

|        | Get's sick  | Does Not  |
|---     |:-:          | :-:|
|Not Treated | a           | b  |
|Treated     | c           | d  |


$$ Relative Risk = \frac{ \frac{a}{a+b} }{ \frac{c}{c+d}    }$$


$$ Odds Ratio = \frac{ \frac{a}{b}    }{\frac{c}{d}    } = \frac{ad}{bc} $$


Let's calculate both of those for the AZT data.  (Notice that the columns and rows have been transposed in this example, so we have to think carefully about what values should be used; this is one reason why considering terms like *false positive*, *true positive*, *false negative* and *true negative* can be helpful.)



|      |AZT : Yes |AZT : No  |
|---   |:-:      |:-:      |
|sick  |25       |124      |
|not   |145      |44       |


$$Rel. Risk = \frac{ \frac{124}{124+44}  }{ \frac{25}{25+145}}   \approx 5.02  $$

So, *if* a person had HIV and choose not to take AZT, then that person was apparently 5 times more likely to develop AIDS.  


$$ Odds Ratio = \frac{124 \cdot 145}{25 \cdot 44} \approx 16.34 $$

Now, the *odds* of developing AIDS when not taking AZT are 16 times higher then when taking AZT.  Odds are not as useful, or as easy to understand, as the relative risk.  That being said, the odds ratio (OR) is more standard in many fields.  

With large populations and rare diseases, the odds ratio and relative risk are approximately equal.   

Consider this fictious example

|      |Trmt : Yes |Trmt : No  |
|---   |:-:      |:-:          |
|sick  |40       |10           |
|not   |600      |500          |


$$RR = \frac{\frac{40}{40+600} }{\frac{10}{10+500}} \approx 3.19 $$

$$OR = \frac{40 \cdot 600}{10 \cdot 500} \approx 3.33 $$


Notice that the relative risk and the odds ratio are similar.  The fact that the odds ratio is often used to approximate the relative risk is one of the reasons that people will occassionally misstate the definition of the odds ratio (stating the correct definition of the relative risk instead).  


If $\pi_1$ is the probability becoming sick without treatment and $\pi_2$ is the probability of becoming sick with the treatment.  Then $$ RR = \frac{\pi_1}{\pi_2} ,$$ and is interpretted as how many times more likely a person is to become sick if he or she foregoes the treatment.  

In the same terms, the odds ratio is 

$$ OR = \frac{\frac{\pi_1}{1-\pi_1}}{\frac{\pi_2}{1-\pi_2}}$$

$$ OR = \frac{\pi_1/(1-\pi_1)}{\pi_2/(1-\pi_2)}=\frac{\pi_1 \cdot (1-\pi_2)}{\pi_2 \cdot (1-\pi_1)}$$

Notice this:

$$ OR = RR \cdot \frac{(1-\pi_2)}{(1-\pi_1)}$$



In a future notebook, we will learn other interesting facts about the odds ratio.  



## Rates, Trues, Falses, Positives and Negatives

|        |Test Pos. | Test Neg. |
|---     |:-:       |:-:        |
|Sick    |a        |b         |
|Not Sick|c        |d          |


Or

|        |Test Pos. | Test Neg. |
|---     |:-:       |:-:        |
|Sick    |True Pos. |False Neg. |
|Not Sick|False Pos.|True Neg.   |




The False Positive Rate is $$ FP = \frac{c}{a+c} = \frac{False\ Pos.}{All\ Pos.}$$

The True Positive Rate is $$ TP = 1- FP = \frac{a}{a+c} = \frac{True\ Pos.}{All\ Pos.}$$

The False Negative Rate is $$ FP = \frac{b}{b+d} = \frac{False\ Neg.}{All\ Neg.}$$

The True Negative Rate is $$ TP = 1- FP = \frac{d}{b+d} = \frac{True\ Neg.}{All\ Neg.}$$


Accuracy is $$Acc = \frac{a + d}{a+b+c+d} = \frac{Trues}{Total}$$

Consider this data on mushrooms, where under Poisonous the response of p means poisonous and e means edible.  




In [59]:
mushrooms =Table.read_table("mushrooms.csv")

mushrooms.show(5)

Poisonous,Cap Shape,Cap Surface,Cap Color,Bruises,Odor,Gill Attachment,Gill Spacing,Gill Size,Gill Color,Stalk Shape,Stalk Root,Stalk surface above ring,Stalk surface below ring,Stalk color above ring,Stalk color below ring,Veil type,Veil color,Ring number,Ring type,Spore print color,Population,Habit
p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g


In [7]:
2752/(2752+ 624)

0.8151658767772512

In [8]:
3292/(1456+3292)

0.6933445661331087

In [60]:
mushrooms.pivot("Poisonous", "Bruises")

Bruises,e,p
f,1456,3292
t,2752,624


This is a **very bad idea** (do not eat wild mushrooms unless you really know what they are) but suppose we used Bruises as a predictor for whether the mushroom was edible.  (So, bruises means we'll label it edible.)

a) What is the Sensitivity of this?

b) Specificity?

c) Relative risk?  For this, to make it make sense, let's compute the risk of eating a poisonous mushroom if bruises were your only test.  

d) Odd ratio?  Same as RR  

e) False positive rate?

d) Accuracy?

|Test | Edible |Poisonous|Total|
|:-:|:-:|:-:|--:|
| F |1456|3292|4748|
| T |2752|624|3376|
|Total|4208|3916|8124|

Look at it this way:

|Test | Edible |Poisonous|
|:-:|:-:|:-:|
| F |False Neg|True Neg|
| T |True Pos|False Pos|

It helps to stop and relabel the confusion matrix just to be certain that we are all considering the same outcomes in the same way.


In [61]:
sens = ...  

sens

Ellipsis

In [62]:
spec = ...

spec

Ellipsis

In [63]:
RR = ...

RR

Ellipsis

In [64]:
OR = ...

OR

Ellipsis

In [65]:
FP = ...

FP

Ellipsis

In [66]:
Acc = ...

Acc

Ellipsis

In [67]:
## Cheating a little and working them out ahead of time.  

sens = 2752/4208
spec = 3292/3916
pi1 = 2752/(2752+624)
pi2 = 1456/(1456+3292)

RR = pi1/pi2

OR = RR *(1-pi2)/(1-pi1)

FP = 664/3376

Acc = (3292 + 2752)/8124

print(f"sens = {sens}\nspec = {spec} \nRR = {RR} \nOR = {OR} \nFP = {FP} \nAcc = {Acc}")


sens = 0.6539923954372624
spec = 0.8406537282941777 
RR = 2.6582469663038384 
OR = 9.971541279233586 
FP = 0.1966824644549763 
Acc = 0.7439684884293452


Are there any other variables in this data set that are "better" predictors for whether a mushroom is edible?

There are other, binary variables in this data set.  We will revisit this data set when we study classification.  


### Defining a function we can copy and use whenever needed.  

Arrange your input table in this order.  

|        |Test Pos. | Test Neg. |
|---     |:-:       |:-:        |
|Sick    |a        |b         |
|Not Sick|c        |d          |

Or,

|        |Test Pos. | Test Neg. |
|---     |:-:       |:-:        |
|Sick    |True Pos. |False Neg. |
|Not Sick|False Pos.|True Neg.   |

In [16]:
def TwoWaySummary(x):
    """ x must be a 2x2 table arranged T+ F-, F+ T-"""
    a = x[[0],[0]][0]
    b = x[[0],[1]][0]
    c = x[[1],[0]][0]
    d = x[[1],[1]][0]
    
    print(f"sensitivity = {a/(a+b)}\nspecificity = {d/(c+d)}\nrelative risk = {a*(c+d)/(c*(a+b))}\nodds ratio = {a*d/(b*c)}\naccuracy = {(a+d)/(a+d+c+b)}")
    


In [17]:
TwoWaySummary(np.array([[1, 2], [3, 4]]))

sensitivity = 0.3333333333333333
specificity = 0.5714285714285714
relative risk = 0.7777777777777778
odds ratio = 0.6666666666666666
accuracy = 0.5
